In [2]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
import psycopg2
import pandas as pd

In [5]:
df = pd.read_excel('SuperStoreUS-2015.xlsx')
df.head()

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,...,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
0,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2015-01-07,2015-01-08,4.5600,4,13.01,88522
1,20228,Not Specified,0.02,500.98,26.00,5,Ronnie Proctor,Delivery Truck,Home Office,Furniture,...,West,California,San Gabriel,91776,2015-06-13,2015-06-15,4390.3665,12,6362.85,90193
2,21776,Critical,0.06,9.48,7.29,11,Marcus Dunlap,Regular Air,Home Office,Furniture,...,East,New Jersey,Roselle,7203,2015-02-15,2015-02-17,-53.8096,22,211.15,90192
3,24844,Medium,0.09,78.69,19.99,14,Gwendolyn F Tyson,Regular Air,Small Business,Furniture,...,Central,Minnesota,Prior Lake,55372,2015-05-12,2015-05-14,803.4705,16,1164.45,86838
4,24846,Medium,0.08,3.28,2.31,14,Gwendolyn F Tyson,Regular Air,Small Business,Office Supplies,...,Central,Minnesota,Prior Lake,55372,2015-05-12,2015-05-13,-24.0300,7,22.23,86838


In [6]:
def create_database():
    try:
        conn = psycopg2.connect(   #for connection
            host="localhost",
            user="postgres",
            password="admin"
        )
        conn.autocommit = True
        cur = conn.cursor() #for execution of query

        cur.execute("CREATE DATABASE superstoreus_2015_db;")
        print("Database created successfully!")
        
        cur.close()
        conn.close()
    except Exception as e:
        print(f"Error: {e}")
        
create_database()

Database created successfully!


In [7]:
def create_tables():
    try:
        conn = psycopg2.connect(
            dbname="superstoreus_2015_db",
            user="postgres",
            password="admin",
            host="localhost"
        )
        cur = conn.cursor()

        # creating cx table
        cur.execute('''
        CREATE TABLE IF NOT EXISTS Customers (
            cxID INT PRIMARY KEY,          -- Customer ID will be the primary key
            cxName VARCHAR(255),
            cxSegment VARCHAR(255),
            country VARCHAR(255),
            region VARCHAR(255),
            state VARCHAR(255),
            city VARCHAR(255),
            postalCode VARCHAR(20)
        );
        ''')

        # creating Products table
        cur.execute('''
        CREATE TABLE IF NOT EXISTS Products (
            productName VARCHAR(255),
            productCategory VARCHAR(255),
            productSubCategory VARCHAR(255),
            productContainer VARCHAR(255),
            productBaseMargin FLOAT,
            unitPrice FLOAT,
            PRIMARY KEY (productName, productCategory, productSubCategory)   -- Composite key
        );
        ''')

        # creating Orders table
        cur.execute('''
        CREATE TABLE IF NOT EXISTS Orders (
            orderID INT,
            orderDate DATE,
            shipDate DATE,
            orderPriority VARCHAR(50),
            shippingCost FLOAT,
            sales FLOAT,
            profit FLOAT,
            quantityOrdered INT,
            discount FLOAT,
            shipMode VARCHAR(255),
            cxID INT,  -- Foreign key to Customers
            productName VARCHAR(255),
            productCategory VARCHAR(255),
            productSubCategory VARCHAR(255),
            PRIMARY KEY (orderID, cxID, productName),   -- Composite key
            FOREIGN KEY (cxID) REFERENCES Customers(cxID),
            FOREIGN KEY (productName, productCategory, productSubCategory) REFERENCES Products(productName, productCategory, productSubCategory)
        );
        ''')

        conn.commit()
        cur.close()
        conn.close()
        print("Tables created successfully!")
    except Exception as e:
        print(f"Error: {e}")
        
create_tables()

Tables created successfully!


In [8]:
def insert_data_from_csv(excel_file):
    try:
        conn = psycopg2.connect(
            dbname="superstoreus_2015_db",
            user="postgres",
            password="admin",
            host="localhost"
        )
        cur = conn.cursor()

        # Loading data from csv file into Pandas DataFrame
        df = pd.read_excel('SuperStoreUS-2015.xlsx')

        # Cx data
        for index, row in df.iterrows():
            cur.execute('''
                INSERT INTO Customers (cxID, cxName, cxSegment, country, region, state, city, postalCode)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (cxID) DO NOTHING;
            ''', (row['Customer ID'], row['Customer Name'], row['Customer Segment'], row['Country'], row['Region'], row['State or Province'], row['City'], row['Postal Code']))


        # Products data
        for index, row in df.iterrows():
            cur.execute('''
                INSERT INTO Products (productName, productCategory, productSubCategory, productContainer, productBaseMargin, unitPrice)
                VALUES (%s, %s, %s, %s, %s, %s)
                ON CONFLICT (productName, productCategory, productSubCategory) DO NOTHING;
            ''', (row['Product Name'], row['Product Category'], row['Product Sub-Category'], row['Product Container'], row['Product Base Margin'], row['Unit Price']))


        # Orders data
        for index, row in df.iterrows():
            cur.execute('''
                INSERT INTO Orders (orderID, orderDate, shipDate, orderPriority, shippingCost, sales, profit, quantityOrdered, discount, shipMode, cxID, productName, productCategory, productSubCategory)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ''', (row['Order ID'], row['Order Date'], row['Ship Date'], row['Order Priority'], row['Shipping Cost'], row['Sales'], row['Profit'], row['Quantity ordered new'], row['Discount'], row['Ship Mode'], row['Customer ID'], row['Product Name'], row['Product Category'], row['Product Sub-Category']))

        conn.commit()
        cur.close()
        conn.close()
        print("Data inserted successfully!")
    except Exception as e:
        print(f"Error: {e}")

insert_data_from_csv('SuperStoreUS-2015.xlsx')  

Data inserted successfully!
